In [1]:
# do new env 
#Tested on Python >=3.9
import pandas as pd
import base64
from openai import OpenAI
import time
key= 'Your-Key-Here'
client = OpenAI(api_key=key)


In [2]:

def get_stratified_sample(df, difficulty, specialty=None):
    # Use exception handling to catch errors depending on your use Case
    # Check if necessary columns are present in DataFrame
    required_columns = {'Specialty', 'CorrectResponse', 'Questions'}
    if not required_columns.issubset(df.columns):
        print("Error: Missing required columns in DataFrame. Required columns are: 'Specialty', 'CorrectResponse', 'Questions'")
        # raise ValueError("Missing required columns in DataFrame. Required columns are: 'Specialty', 'CorrectResponse', 'Questions'")
        return None

    # Handling different difficulty levels
    if difficulty == 'Alpha':
        if specialty is None:
            print("Error: Specialty must be specified for difficulty 'Alpha'")
            # raise ValueError("Specialty must be specified for difficulty 'Alpha'")
            return None
        if specialty not in df['Specialty'].unique():
            print(f"Error: Specialty '{specialty}' not found in DataFrame")
            # raise ValueError(f"Specialty '{specialty}' not found in DataFrame")
            return None
        
        # Filter DataFrame for the given specialty
        specialty_df = df[df['Specialty'] == specialty]
        correct_responses = specialty_df['CorrectResponse']
        questions = specialty_df['Questions']
        return specialty_df, correct_responses, questions

    elif difficulty == 'Beta':
        # Perform stratified sampling to retrieve 50% from each specialty
        try:
            selected = df.groupby('Specialty', group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=42))
        except ValueError as e:
            print(f"Error during sampling: {e}")
            return None
        
        correct_responses = selected['CorrectResponse']
        questions = selected['Questions']
        return selected, correct_responses, questions

    elif difficulty == 'Production':
        # Return the entire DataFrame, ie the Full Dataset
        questions = df['Questions']
        correct_responses = df['CorrectResponse']
        return df, correct_responses, questions

    else:
        print("Error: Invalid difficulty level. Expected 'Alpha', 'Beta', or 'Production'")
        return None


In [3]:
def chat_with_assistant(user_input,model='OpenAI'):
    # Custom tools can be defined here (if needed)
  if model =='OpenAI':
      tools = [
          {
            "type": "function",
            "function": {
                "name":"Assistant",
              "description": '''On Multiple choice Quiz choose correct responses:(ONLY THE CORRECT LETTERS and no spaces and no other associated Text. If more than one letter then add a dash- between letters).
              For example if question is "Which of the following are programming languages? A. Python B. HTML C. JavaScript D. SQL E. CSS. then your response should be:A-C-D'''
            }
          }
        ]
        
      
      # Check if there's an existing conversation history
      if 'history' not in chat_with_assistant.__dict__:
          chat_with_assistant.history = []

      # Append the user's message to the history
      chat_with_assistant.history.append({"role": "user", "content": user_input})

      # Generate a response from the assistant
      completion = client.chat.completions.create(
        model="gpt-4-0613",
        messages=chat_with_assistant.history,
        # stream=True,
        tools=tools,
        tool_choice="none",
      )
      full_response = completion.choices[0].message.content
      
      chat_with_assistant.history = []
      #Here add to get
      return full_response
  else:
      #  Add your model logic here 
      return "Your Model's response"


In [3]:
#Setting up the collection of questions and response handling
import pandas as pd
# load from local file if downloaded or use hGGimport.py to load from huggingFace and add your logic
Cognet=pd.read_csv('cgnetMD1_0.csv') 
# Sample the DataFrame and choose difficulty level as 'Alpha', 'Beta', or 'Production'
# If Alpha is chosen, specify the specialty as well -At version 1.0 available specialties are:  'Dermatology', 'Psychiatry', 'Neurology', 'Endocrinology', 'Pulmonology-Respiratory'
sampled_df, correct_responses,questions = get_stratified_sample(Cognet,'Beta')

# Print for testing purposes-Consider computational cost
# print(sampled_df['Specialty'].value_counts())
# print(correct_responses.tolist())
# print(questions.tolist())

,Questions,CorrectResponse,Specialty
0,Which of the following is not identified as a ...,D,Psychiatry
1,Which term refers to the likelihood of identif...,E,Psychiatry
2,What is the probability of a patient inheritin...,A,Psychiatry
3,Is the term used to describe a situation where...,B,Psychiatry
4,Which of the following is not a characteristic...,B,Psychiatry


In [ ]:
# Test if it works 
chat_with_assistant("Which of the following are programming languages? A. Python B. HTML C. JavaScript D. SQL E. CSS. ")

In [ ]:
# Depending on Size of dataframe add delay as sleep to avoid rate limit or buffer overflow
# Could be adjusted to handle the rate limit via a try except block and sleep for a few seconds, but it should work fine testing it out
#Adjust Sleep time as needed depending on your model and rate limit
collectResponses=[]
for i in questions:
    Response = chat_with_assistant(i)
    time.sleep(1)
    print(Response)
    collectResponses.append(Response)


<h3>Clean up quiz</h3>

In [ ]:
import re

# Clean up to ensure the quiz list is in the correct format Adjust as required based on the response from the assistant
def clean_and_order_quiz_list(quiz_list, separator='-'):
    cleaned_list = []
    counter =0

    for item in quiz_list:
        # Check if the item contains letters with associated text
        if re.search(r'[A-Z]\.', item):
            # Extract letter choices and sort them
            choices = re.findall(r'([A-Z])\.', item)
            #Count the instances where LLM provided a wrong response structure
            counter+=1
            # Sort the choices
            print(counter)
            sorted_choices = sorted(choices)
            # Join the sorted choices with dashes
            cleaned_item = separator.join(sorted_choices)
        else:
            # Ensure item is in the correct format (choices separated by dashes)
            # and sort them if it's a single letter
            choices = sorted(item.split(separator))
            cleaned_item = separator.join(choices)

        cleaned_list.append(cleaned_item)

    return cleaned_list



In [ ]:
quiz_list = collectResponses
print(collectResponses)


In [ ]:
# COGNET-MD v.1.0 - Score Responses
def score_responses(user_responses, correct_responses):
    score = 0.0
    
   
    # to distict between normal and hard rules of conduct are either suggestive or not suggestive
    PartialCredit=0
    FullCredit=0
    penaltyCount = 0
    
    partialCreditIndex = []
    fullCreditIndex = []
    for index, (user_response, correct_response) in enumerate(zip(user_responses, correct_responses)):
            # Split the responses into sets of choices
        user_choices = set(user_response.split('-'))
        correct_choices = set(correct_response.split('-'))
            
            # Calculate correct and incorrect choices
        correct_selected = user_choices & correct_choices
        incorrect_selected = user_choices - correct_choices
        
        #count correct and incorrect selectec
        
        if correct_selected:
            partialCreditIndex.append(index)
                # Partial credit for selecting at least one correct answer
            PartialCredit+=1
            score += 0.5
            
        if correct_selected == correct_choices and not incorrect_selected:
                # Full credit for selecting all correct answers and no incorrect ones
            fullCreditIndex.append(index)
            FullCredit+=1
            score += 0.5  # additional 0.5 for making it full 1 point in total
            
            # Deduct points for incorrect answers
        if incorrect_selected:
            score -= 0.5 * len(incorrect_selected)
            penaltyCount += len(incorrect_selected)  # Count each incorrect choice as a penalty

    return PartialCredit,FullCredit,score,partialCreditIndex,fullCreditIndex,penaltyCount

def calculate_accuracy(user_responses, correct_responses):
    total_questions = len(correct_responses)
    
    max_score = total_questions  # Each question is worth 1 point if answered correctly
    PartialCredit,FullCredit,actual_score,partialCreditIndex,fullCreditIndex,penaltyCount = score_responses(user_responses, correct_responses)
    print(actual_score)
    accuracy = (actual_score/ max_score) * 100
    return PartialCredit,FullCredit,accuracy,partialCreditIndex,fullCreditIndex,penaltyCount


In [ ]:
cleaned_quiz_list=clean_and_order_quiz_list(quiz_list)
print(cleaned_quiz_list)
print(correct_responses.tolist())

In [ ]:
PartialCredit,FullCredit,accuracy,partialCreditIndex,fullCreditIndex,penaltyCount= calculate_accuracy(cleaned_quiz_list, correct_responses.tolist())
print(f"Partial Credits were equal to: {PartialCredit*0.5}, for at least one correct choice in Multiple choice questions ")
print(f"Total partially answered questions: {PartialCredit}")
print("Full Credit, all correct in a Multiple choice question:", FullCredit)

print(f"Penalties given {penaltyCount} times for a total of {penaltyCount*0.5} points being deducted")
print(f"Accuracy Based on COGNET-MD scoring system:\n {accuracy}% for a total of {len(cleaned_quiz_list)} questions")
